In [1]:
from src.collision import CollisionDetector
obj_folder = "/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/segmentation"

In [2]:
t = CollisionDetector(obj_folder=obj_folder, skeleton_toml_path="skeleton.toml", pose_csv="./tracking/28/240905-1616_session28_track99_points3d.csv", obj_ref_frame=2599, obj_transform_toml=f"{obj_folder}/240905-1616_seed_session28_first.toml")

In [ ]:
from src.object import CollisionObj
raw_mesh = CollisionObj(obj_folder=obj_folder)

In [3]:
from src.animal import AnimalList
track_folder = "/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/tracking"
A = AnimalList("skeleton.toml", track_folder, session_number=28)

PosixPath('/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/tracking/240905-1616_session28_track0_points3d.csv') generated an exception: Pose csv does not contain any valid frames
PosixPath('/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/tracking/240905-1616_session28_track11_points3d.csv') generated an exception: Pose csv does not contain any valid frames
PosixPath('/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/tracking/240905-1616_session28_track10_points3d.csv') generated an exception: Pose csv does not contain any valid frames
PosixPath('/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/tracking/240905-1616_session28_track12_points3d.csv') generated an exception: Pose csv does not contain any valid frames
PosixPath('/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/tracking/240905-1616_session28_track13_points3d.csv') generated an exception: Pose csv does not contain any valid frames
PosixPath('/Users/persie/PhD_Code/3d_ant_data_

In [9]:
from src.visual_widget import MultiViewer
V = MultiViewer(A.animals, [t.obj], 0)

KeyboardInterrupt: 

In [ ]:
V._scene_widget.scene.graph.show()